In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

In [3]:
batch_size = 50
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Before you can train your model, 
sequences = []
with open("emi_R4_neg_seqs_1NotR_3.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            sequences.append(seq)
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

In [5]:
## 
average_hidden_list = []
final_hidden_list = []
hs_list = []
final_cell_list = []


num2 = range(0, 50)
x = 0
y = 50
for i in num2:
    num1 = range(x, y)
    for j in num1:
        avg_hidden, final_hidden, final_cell, hs_out = (b.get_rep_hs(sequences[j]))
        average_hidden_list.append(avg_hidden)
        final_hidden_list.append(final_hidden)
        final_cell_list.append(final_cell)
        hs_list.append(hs_out)
    print('rep')
    x = x + 50
    y = y + 50
    


rep
rep
rep
rep
rep
rep
rep
rep
rep
rep


IndexError: list index out of range

In [6]:
average_hidden_pd = pd.DataFrame(np.row_stack(average_hidden_list))
final_hidden_pd = pd.DataFrame(np.row_stack(final_hidden_list))
hidden_state = pd.DataFrame(np.row_stack(hs_list))
print(average_hidden_pd)

           0         1         2         3         4         5         6   \
0    0.020252  0.157824 -0.094644 -0.961551 -0.011939 -0.215804  0.061780   
1    0.033988  0.161367 -0.095499 -0.952809 -0.006059 -0.255837  0.065383   
2    0.023008  0.160202 -0.102350 -0.959873 -0.015605 -0.230402  0.069177   
3    0.026343  0.164728 -0.102564 -0.957004 -0.013024 -0.229810  0.063702   
4    0.032950  0.171054 -0.093357 -0.958912 -0.011040 -0.232875  0.064333   
5    0.040884  0.176493 -0.092444 -0.956522 -0.008929 -0.228047  0.060985   
6    0.028370  0.163773 -0.097766 -0.957103 -0.012165 -0.223984  0.061115   
7    0.031212  0.169086 -0.096837 -0.957534 -0.011437 -0.233952  0.065599   
8    0.036384  0.162148 -0.101297 -0.956182 -0.012831 -0.254161  0.065732   
9    0.009309  0.155546 -0.023036 -0.949266 -0.009048 -0.147732  0.076427   
10   0.027862  0.156855 -0.097674 -0.953774 -0.006963 -0.276990  0.069699   
11   0.032171  0.170128 -0.101071 -0.956593 -0.012899 -0.252840  0.067143   

In [7]:
average_hidden_pd.to_csv("emi_R4_neg_reps_1NotR_3.csv")
#final_hidden_pd.to_csv("scp_finalhidden_neg_3.csv")

In [7]:
import pickle
save_loc = "C:\\Users\\pkinn\\Documents\\UniRep\\full representations\\emi larger set\\"
data_name = 'emi_pos_reps_ova_3'
file_append = '.pickle'


fn = save_loc + data_name + 'avg_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(average_hidden_list, f)

fn = save_loc + data_name + 'final_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_hidden_list, f)

fn = save_loc + data_name + 'final_cell' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_cell_list, f)
   
fn = save_loc + data_name + 'hidden_state' + file_append
with open(fn, 'wb') as f:
    pickle.dump(hs_list, f)
   
fn = save_loc + data_name + 'all_output_hs' + file_append
with open(fn, 'wb') as f:
    pickle.dump([average_hidden_list, final_hidden_list, final_cell_list, hs_list], f)